# import modules

In [1]:
import pandas as pd
import numpy as np
import transliterate
from sklearn import linear_model
import joblib
import plotly.express as px
from IPython.core.display import display, HTML
from sklearn.ensemble import RandomForestRegressor
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error,r2_score
import seaborn as sns
import lightgbm as lgb
import re
reg = re.compile('[^a-zA-Z0-9]')
import joblib
import plotly.graph_objects as go
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
display(HTML("<style>.container { width:90% !important; }</style>"))
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 1000)

In [2]:
# загрузка функций
def reduce_mem_usage(df):
    return df
# перевод названия параметров в латиницу
def rename_df(data):
    data_name=[]
    for i in data.columns:
        try:
            data_name.append(reg.sub('', transliterate.translit(str(i), reversed=True)))
        except:
            data_name.append(reg.sub('', str(i)))
    data.columns=data_name
    return(data)

# read data

In [3]:
gmc_al=pd.read_csv('GMC_al.csv',low_memory=False)
gmc_red_ch=pd.read_csv('GMC_red_ch.csv',low_memory=False)
OM=pd.read_excel('../Объектная модель ПАЗ.xlsx',sheet_name='сценарий 3')
OM=rename_df(OM)

OM.columns=OM.columns.str.lower()
OM['tselevojparametr']=OM['tselevojparametr'].str.lower()

In [4]:
# преобразование данных
gmc_red_ch['Дозировка оборотного раствора ГМЦ-1']=gmc_red_ch['Дозировка оборотного раствора ГМЦ-1']*gmc_red_ch['Расход боксита по конвейеру ГМЦ-1']
gmc_al['Дозировка оборотного раствора ГМЦ-1']=gmc_al['Дозировка оборотного раствора ГМЦ-1']*gmc_al['Расход боксита по конвейеру ГМЦ-1']
gmc_red_ch['Расход оборотного раствора ЦС4']=gmc_red_ch['Расход оборотного раствора ЦС4']+gmc_red_ch['Расход фактический маточного раствора общий ЦС4']
gmc_al['606_Vlaga']=gmc_al['Расход боксита по конвейеру ГМЦ-1']*gmc_al['606_Vlaga']/100
gmc_al['606_Fe2O3']=gmc_al['Расход боксита по конвейеру ГМЦ-1']*gmc_al['606_Fe2O3']/100
gmc_al['606_Al2O3']=gmc_al['Расход боксита по конвейеру ГМЦ-1']*gmc_al['606_Al2O3']/100
gmc_al['606_C_opg']=gmc_al['Расход боксита по конвейеру ГМЦ-1']*gmc_al['606_C_opg']/100
gmc_al['606_SiO2']=gmc_al['Расход боксита по конвейеру ГМЦ-1']*gmc_al['606_SiO2']/100
gmc_al['606_CO2']=gmc_al['Расход боксита по конвейеру ГМЦ-1']*gmc_al['606_CO2']/100
gmc_al['606_CaO']=gmc_al['Расход боксита по конвейеру ГМЦ-1']*gmc_al['606_CaO']/100

gmc_red_ch['606_Vlaga']=gmc_red_ch['Расход боксита по конвейеру ГМЦ-1']*gmc_red_ch['606_Vlaga']/100
gmc_red_ch['606_Fe2O3']=gmc_red_ch['Расход боксита по конвейеру ГМЦ-1']*gmc_red_ch['606_Fe2O3']/100
gmc_red_ch['606_Al2O3']=gmc_red_ch['Расход боксита по конвейеру ГМЦ-1']*gmc_red_ch['606_Al2O3']/100
gmc_red_ch['606_C_opg']=gmc_red_ch['Расход боксита по конвейеру ГМЦ-1']*gmc_red_ch['606_C_opg']/100
gmc_red_ch['606_SiO2']=gmc_red_ch['Расход боксита по конвейеру ГМЦ-1']*gmc_red_ch['606_SiO2']/100
gmc_red_ch['606_CO2']=gmc_red_ch['Расход боксита по конвейеру ГМЦ-1']*gmc_red_ch['606_CO2']/100
gmc_red_ch['606_CaO']=gmc_red_ch['Расход боксита по конвейеру ГМЦ-1']*gmc_red_ch['606_CaO']/100

In [5]:
# перемеинование столбцов
gmc_al=rename_df(gmc_al)
gmc_red_ch=rename_df(gmc_red_ch)
gmc_al.columns=gmc_al.columns.str.lower()
gmc_red_ch.columns=gmc_red_ch.columns.str.lower()

In [6]:
# перемеинование столбцов
for i in range(OM.shape[0]):
    try:
        OM['tselevojparametr'][i]=reg.sub('',transliterate.translit(str(OM['tselevojparametr'][i]),reversed=True))
    except:
        OM['tselevojparametr'][i]=reg.sub('',OM['tselevojparametr'][i])
OM.loc[OM['tselevojparametr']=='1337sopg','tselevojparametr']='1337copg'

# feature engineering

In [7]:
# преобразование даты
gmc_al['date']=pd.to_datetime(gmc_al['date'], format='%Y-%m-%d %H:%M:%S')
gmc_red_ch['date']=pd.to_datetime(gmc_red_ch['date'], format='%Y-%m-%d %H:%M:%S')
gmc_red_ch2=gmc_red_ch[gmc_red_ch['date']<'2019']
gmc_red_ch=gmc_red_ch[gmc_red_ch['date']>='2019']

In [8]:
# убираем индексы
gmc_al=gmc_al.reset_index(drop=True)
gmc_red_ch=gmc_red_ch.reset_index(drop=True)
gmc_red_ch2=gmc_red_ch2.reset_index(drop=True)

In [9]:
# преобразование данных
gmc_al['606modkp']=gmc_al['606al2o3']/gmc_al['606sio2']
gmc_red_ch['606modkp']=gmc_red_ch['606al2o3']/gmc_red_ch['606sio2']
gmc_red_ch2['606modkp']=gmc_red_ch2['606al2o3']/gmc_red_ch2['606sio2']
gmc_al['606al2o3']=gmc_al['606al2o3']+gmc_al['1064al2o3']/1000*gmc_al['dozirovkaoborotnogorastvoragmts1']
gmc_red_ch['606al2o3']=gmc_red_ch['606al2o3']+gmc_red_ch['1064al2o3']/1000*gmc_red_ch['dozirovkaoborotnogorastvoragmts1']
gmc_red_ch2['606al2o3']=gmc_red_ch2['606al2o3']+gmc_red_ch2['1064al2o3']/1000*gmc_red_ch2['dozirovkaoborotnogorastvoragmts1']
gmc_al['1064na2oku']=gmc_al['1064na2oku']/1000*gmc_al['dozirovkaoborotnogorastvoragmts1']
gmc_red_ch['1064na2oku']=gmc_red_ch['1064na2oku']/1000*gmc_red_ch['dozirovkaoborotnogorastvoragmts1']
gmc_red_ch2['1064na2oku']=gmc_red_ch2['1064na2oku']/1000*gmc_red_ch2['dozirovkaoborotnogorastvoragmts1']
gmc_red_ch['671cao']=gmc_red_ch['671cao']/100*(gmc_red_ch['rashodsuhogoboksitatss1']+gmc_red_ch['podanoizvestnjakavtss1'])
gmc_red_ch['671sio2']=gmc_red_ch['671sio2']/100*(gmc_red_ch['rashodsuhogoboksitatss1']+gmc_red_ch['podanoizvestnjakavtss1'])
gmc_red_ch['671vlaga']=gmc_red_ch['671vlaga']/100*(gmc_red_ch['rashodsuhogoboksitatss1']+gmc_red_ch['podanoizvestnjakavtss1'])
gmc_red_ch['671al2o3']=gmc_red_ch['671al2o3']/100*(gmc_red_ch['rashodsuhogoboksitatss1']+gmc_red_ch['podanoizvestnjakavtss1'])

gmc_red_ch2['671cao']=gmc_red_ch2['671cao']/100*(gmc_red_ch2['rashodsuhogoboksitatss1']+gmc_red_ch2['podanoizvestnjakavtss1'])
gmc_red_ch2['671sio2']=gmc_red_ch2['671sio2']/100*(gmc_red_ch2['rashodsuhogoboksitatss1']+gmc_red_ch2['podanoizvestnjakavtss1'])
gmc_red_ch2['671vlaga']=gmc_red_ch2['671vlaga']/100*(gmc_red_ch2['rashodsuhogoboksitatss1']+gmc_red_ch2['podanoizvestnjakavtss1'])
gmc_red_ch2['671al2o3']=gmc_red_ch2['671al2o3']/100*(gmc_red_ch2['rashodsuhogoboksitatss1']+gmc_red_ch2['podanoizvestnjakavtss1'])

In [10]:
%%time
# обучение моделей
data_model_coef=pd.DataFrame(columns=list(OM.columns))
data_model_coef=data_model_coef.rename(columns={'stsenarij':'intercept'})
for i in range(OM.shape[0]):
#     y=gmc[(~gmc[OM['tselevojparametr'][i]].isna()) & (gmc[OM['tselevojparametr'][i]]!=0)][OM['tselevojparametr'][i]]
    x_list=OM.loc[OM['tselevojparametr']==OM['tselevojparametr'][i]].iloc[:,4:]
    
    if list(OM.loc[OM['tselevojparametr']==OM['tselevojparametr'][i]]['pp'])[0]=='ch':
        X=gmc_red_ch[(~gmc_red_ch[OM['tselevojparametr'][i]].isna()) & (gmc_red_ch[OM['tselevojparametr'][i]]>0.001)][[OM['tselevojparametr'][i]]+list(x_list.loc[:, (x_list != 0).any(axis=0)].columns)]
    elif list(OM.loc[OM['tselevojparametr']==OM['tselevojparametr'][i]]['pp'])[0]=='al':
        X=gmc_al[(~gmc_al[OM['tselevojparametr'][i]].isna()) & (gmc_al[OM['tselevojparametr'][i]]>0.001)][[OM['tselevojparametr'][i]]+list(x_list.loc[:, (x_list != 0).any(axis=0)].columns)]
    else:
        X=gmc_red_ch2[(~gmc_red_ch2[OM['tselevojparametr'][i]].isna()) & (gmc_red_ch2[OM['tselevojparametr'][i]]>0.001)][[OM['tselevojparametr'][i]]+list(x_list.loc[:, (x_list != 0).any(axis=0)].columns)]


#     X=gmc[(~gmc[OM['tselevojparametr'][i]].isna()) & (gmc[OM['tselevojparametr'][i]]>0.001)][[OM['tselevojparametr'][i]]+list(x_list.loc[:, (x_list != 0).any(axis=0)].columns)]
    X.dropna(inplace=True)
    y=X[[OM['tselevojparametr'][i]]]
    X=X.drop(columns=[OM['tselevojparametr'][i]])
    
    if OM['tselevojparametr'][i]=='1176al2o3':
        X=X.iloc[900:1000]
        y=y.iloc[900:1000]
        model = linear_model.Ridge(alpha=1.0,normalize=False)
        model.fit(X, y)
    elif OM['tselevojparametr'][i]=='rashodalrranadeksuchizmzapasavm582tss4':
#         X=X.reset_index()
#         y=y.reset_index()
#         X=X[X['index'].isin(list_index)].drop(columns=['index'])
#         y=y[y['index'].isin(list_index)].drop(columns=['index'])
#         print(X.shape)
#         model = linear_model.Ridge(alpha=1.0,normalize=True)
#         model.fit(X, y)
        print('используй модель, как в 2018')
    elif OM['tselevojparametr'][i]=='1plotnostotn':
        X=X.drop(list(pd.DataFrame(y)[pd.DataFrame(y)['1plotnostotn']>5].index)[0],axis=0)
        y=y.drop(list(pd.DataFrame(y)[pd.DataFrame(y)['1plotnostotn']>5].index)[0],axis=0)
        model = linear_model.Ridge(alpha=1.0,normalize=False)
        model.fit(X, y)
    else:
        model = linear_model.Ridge(alpha=1.0)
        model.fit(X, y)
    
    if OM['tselevojparametr'][i]!='rashodalrranadeksuchizmzapasavm582tss4':
        joblib.dump(model, '../model_2018/PAZ_Ridge_'+str(list(y.columns)[0])+'.pkl')
    
        q1=pd.DataFrame(zip(list(X.columns),list(model.coef_[0])), columns=['index','coef']).T
        q1.columns=q1.iloc[0]
        q1=q1.iloc[1:]
        q1['intercept']=list(model.intercept_)[0]
        q1['tselevojparametr']=OM['tselevojparametr'][i]
        q1['pp']=model.score(X, y)
        q1['gmts']=list((abs(model.predict(X) - y)/model.predict(X)*100).mean())[0]
        data_model_coef=pd.concat([data_model_coef,q1])
#     print('coefficient of determination: ', model.score(X, y), '\n')
#     print('mape: ', (abs(model.predict(X) - y)/model.predict(X)*100).mean(),'\n')
#     print('intercept: ',model.intercept_, '\n')
#     print('coef: ',model.coef_,'\n','\n')
    
#     for ii in range(len(X.columns)):
#         print(X.columns[ii], model.coef_[0][ii])

используй модель, как в 2018
CPU times: user 19.1 s, sys: 1.81 s, total: 20.9 s
Wall time: 11.3 s


In [56]:
# сбор екселя с коэффицентами 
OM_1=pd.read_excel('../Объектная модель ПАЗ.xlsx',sheet_name='Прогноз')
OM_1=OM_1[OM_1['Целевой параметр']!='Расход ал.р-ра на дек. с уч. изм. запаса в м.582 ЦС4']
data_model_coef.columns=list(OM_1.columns)
data_model_coef.rename(columns={'сценарий':'intercept'},inplace=True)
data_model_coef.rename(columns={'№ п/п':'score'},inplace=True)
data_model_coef.rename(columns={'№ ГМЦ':'mape'},inplace=True)
data_model_coef['Целевой параметр']=list(OM_1['Целевой параметр'])
data_model_coef[data_model_coef['Целевой параметр']=='Расход ал.р-ра на дек. с уч. изм. запаса в м.582 ЦС4']['671_Al2O3']

In [61]:
data_model_coef.to_excel('data_model_coef_2018.xlsx',index=False)